In [ ]:
from torchvision import transforms, datasets
import torch
import torchvision
import torch.nn as nn
from torch.utils.data import dataset, dataloader
import matplotlib.pyplot as plt
import numpy as np
import torch.nn.functional as F

In [ ]:
class Net(nn.Module):
    height,weight=53,53
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.batch1 = nn.BatchNorm2d(6)
        self.conv2 = nn.Conv2d(6,16, 5)
        self.batch2 = nn.BatchNorm2d(16)
#         self.upsample=nn.ConvTranspose2d(1, 1,20)
        self.fc1 = nn.Linear(1016064, 1000)
        self.fc2 = nn.Linear(1000, 120)
        self.fc3 = nn.Linear(120, 84)
        self.fc4 = nn.Linear(84, 2)
        
    def forward(self, x):

        x = F.relu(self.batch1(self.conv1(x)))
        x = F.relu(self.batch2(self.conv2(x)))
        a=x.cpu()
        plt.imshow(a[0][0].detach().numpy())
        plt.show()
        
        x = x.view(-1, self.num_flat_features(x))
        
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)

        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features
    
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Net().to(device)
print(model)
# import torch
# print(torch.cuda.is_available())

In [ ]:
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

In [ ]:
data_transform = transforms.Compose([
#         transforms.Resize(176),
        transforms.ToTensor()
])

In [ ]:
# train_data = torchvision.datasets.ImageFolder("D:\\Projects\\Alzheimer's Research\\data1\\train", transform=data_transform)
test_data = torchvision.datasets.ImageFolder("resized\\", transform=data_transform)
# train_loader = torch.utils.data.DataLoader(train_data, batch_size=1, shuffle = True, num_workers=4)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=1, shuffle = True, num_workers=4)

In [ ]:
# train_data = torchvision.datasets.ImageFolder("D:\\Projects\\Alzheimer's Research\\data1\\train", transform=data_transform)
val_data = torchvision.datasets.ImageFolder("resized_val", transform=data_transform)
# train_loader = torch.utils.data.DataLoader(train_data, batch_size=1, shuffle = True, num_workers=4)
val_loader = torch.utils.data.DataLoader(test_data, batch_size=1, shuffle = False, num_workers=4)

In [ ]:
test_data.classes

In [ ]:

# net.load_state_dict(torch.load('RecogModelv1.pt'))

In [ ]:
total_step = len(test_loader)
epochs=1
# net.train()
for epoch in range(epochs):
    total=0
    running_loss = 0.0
    for i, (inputs, labels) in enumerate(test_loader):
#         inp = inputs[0][0][5:245,60:300]
#         inp = inp.view(1, 1, 240, 240).to(device)
        inputs=inputs.to(device)
        labels = labels.to(device)
        outputs = model(inputs[0][0].reshape(1,1,inputs.shape[2],inputs.shape[3]))
#         break
#         print(outputs.shape)
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        total=total+1
        print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Total loss : {:.4f}'
            .format(epoch+1,epochs,i+1,total_step, loss.item(), running_loss))
    #         torch.save({'epoch': epoch + 1, 'state_dict': net.state_dict()},
    #                    '{}/checkpoint_{}.pth'.format('nnState', epoch))
    torch.save(model.state_dict(), '..\\Saved Model\\RecogModelv3.pt')

    print("Total Loss : " + str(running_loss/total))
print('Finished Training')

In [ ]:
correct = 0
total = 0
pred=[]
label=[]
with torch.no_grad():
    for images, labels in val_loader:
        inputs=inputs.to(device)
        labels = labels.to(device)
        outputs = model(inputs[0][0].reshape(1,1,inputs.shape[2],inputs.shape[3]))
        print(outputs.shape)
        loss = criterion(outputs, labels)
        print(loss.item())
        _, predicted = torch.max(outputs.data, 1)
        pred.append(predicted)
        label.append(labels)
#         print(predicted.shape)
#         print(labels.shape)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        print('Accuracy of the test images: %d %%' % (100 * correct / total))
        print(predicted,labels)

print('Accuracy of the network on the test images: %d %%' % (
    100 * correct / total))

# print(total)
# print(pred)
# print(label)



In [ ]:
len(pred)
len(label)

In [ ]:
total_step = len(val_loader)
total = 0
correct = 0

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs=inputs.to(device)
        labels = labels.to(device)
        outputs = net(inputs[0][0].reshape(1,1,inputs.shape[2],inputs.shape[3]))
        _, predicted = torch.max(outputs.data, 1)
#         print(predicted, labels)
        total += labels.size(0)
        if predicted == labels :
            correct += 1
# print('Accuracy of the system : {}/{}'.format(correct/total))
#         print()
#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()
#         running_loss += loss.item()
#         total=total+1
#         print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Total loss : {:.4f}'
#             .format(epoch+1,epochs,i+1,total_step, loss.item(), running_loss))
#     print("Total Loss : " + str(running_loss/total))

In [ ]:
torch.save(net.state_dict(), 'Saved Model\\RecogModelv1.pt')

In [ ]:
# torch.save(net, "D:\\Projects\\Alzheimer's Research\\Saved Model\\RecogModel")
torch.save(net.state_dict(), "D:\\Projects\\Alzheimer's Research\\Saved Model\\RecogModel")